In [2]:
import numpy as np
import os
# import six.moves.urllib as urllib
import sys
# import tarfile
# import tensorflow as tf
# import zipfile
import pdb
# from distutils.version import StrictVersion
# from collections import defaultdict
# from io import StringIO
# from matplotlib import pyplot as plt
# from PIL import Image
# import tensorflow.contrib.slim as slim

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
# from object_detection.utils import ops as utils_ops
# from utils import label_map_util
# from utils import visualization_utils as vis_util

# if StrictVersion(tf.__version__) < StrictVersion('1.12.0'):
#   raise ImportError('Please upgrade your TensorFlow installation to v1.12.*.')
# print(tf.__version__)
# This is needed to display the images.
%matplotlib inline

In [9]:
annotation = {}
with open('test_images/train.txt','r') as annotation_file:
    for line in annotation_file:
        split = line.split(' ')
        name = split[0].split('.')[0]
        ground_truth = np.zeros((640, 480))
        for box in split[1:-1]:
            xmin, ymin, xmax, ymax = [int(pix) for pix in box.split(',')[:4]]
            ground_truth[xmin:xmax, ymin:ymax] = 1
        annotation[name] = ground_truth


In [17]:
dir = 'ssd_results/'
for filename in os.listdir(dir):
    if filename.endswith(".npy"): 
        output_dict = np.load(dir + filename).item()
        detection_grid = np.zeros((640, 480))
        name = filename.split('.')[0]
        for i in range(len(output_dict['detection_boxes'])):
            ymin, xmin, ymax, xmax = output_dict['detection_boxes'][i]
            xmin = int(xmin * 640)
            xmax = int(xmax * 640)
            ymin = int(ymin * 480)
            ymax = int(ymax * 480)
            score = output_dict['detection_scores'][i]
            box_grid = detection_grid[xmin:xmax, ymin:ymax]
            box_grid[box_grid < score] = score
            detection_grid[xmin:xmax, ymin:ymax] = box_grid
        # calculate IOU
        intersection = detection_grid[annotation[name] == 1].sum()
        union =  annotation[name].sum() + detection_grid.sum() - intersection
        iou = intersection / union
        print(name, iou)
        

set00_V001_94 0.714875971608496
